In [25]:
import os
from openai import OpenAI
import praw
import prawcore
import pandas as pd

CLIENT_ID = "SkhgKRppbHeDR6HTG4Q1BQ"
CLIENT_SECRET = "daMDLGmHMTPYNIf2t1-Z2s6PXlBtJA"
USER_AGENT = "SentimentChatbot/1.0 (by /u/Affectionate-West-78)"

api_client = praw.Reddit(
    client_id=CLIENT_ID,
    client_secret=CLIENT_SECRET,
    user_agent=USER_AGENT
)

BASE_URL = os.getenv('CS4973_BASE_URL')
API_KEY=api_key=os.getenv('CS4973_API_KEY')
client = OpenAI(base_url=BASE_URL, api_key=API_KEY)


In [16]:
SYSTEM_PROMPT = '''
Come up with a hypothetical question that would be relevant to the university given. 
Return only the question. Be sure to include the name of the university in the question.

Examples:
Question: Boston Univeristy
Answer: How do people feel about the dorms at Boston University?
Question: University of Washington
Answer: At the University of Washington, do people like the quality of the food?
'''

In [28]:
# convert csv to dataframe
df = pd.read_csv('colleges.csv')
df.drop(columns=['location'], inplace=True)

df.head()

,name,subreddit
0,Aberystwyth University,Aberystwyth
1,Acadia University,AcadiaU
2,Algonquin College,Algonquin_College
3,Amarillo College,amarillocollege
4,American University,AmericanU


In [33]:
questions_df = pd.DataFrame(columns=['subreddit', 'question'])
for index, row in df.iterrows():
    messages = [{'role': 'system', 'content': SYSTEM_PROMPT}]
    messages.append({'role': 'user', 'content': row['name']})

    for _ in range(5):
        resp = client.chat.completions.create(
            messages = messages,
            model = "meta-llama/Meta-Llama-3.1-8B-Instruct",
            temperature=0.9,
            max_tokens=500,
        )
        resp = resp.choices[0].message.content
        questions_df.loc[len(questions_df)] = {'subreddit': row['subreddit'], 'question': resp}
questions_df.drop_duplicates(subset=['question'], inplace=True)
questions_df.to_csv('questions.csv', index=False)

KeyboardInterrupt: 

In [36]:
questions_df.to_csv('questions.csv', index=False)

In [38]:
SYSTEM_PROMPT_BENCHMARK = '''
Come up with a hypothetical question that would be relevant to the university given. 
Return only the question. Be sure to include the name of the university in the question.

Examples:
Question: Boston Univeristy
Answer: On a scale of 1-10, how do people feel about the dorms at Boston University?
Question: University of Washington
Answer: On a scale of 1-10, do people like the quality of the food at the University of Washington?
'''

In [39]:
benchmark_df = pd.DataFrame(columns=['subreddit', 'question'])
for index, row in df.iterrows():
    messages = [{'role': 'system', 'content': SYSTEM_PROMPT_BENCHMARK}]
    messages.append({'role': 'user', 'content': row['name']})

    resp = client.chat.completions.create(
        messages = messages,
        model = "meta-llama/Meta-Llama-3.1-8B-Instruct",
        temperature=0.9,
        max_tokens=500,
    )
    resp = resp.choices[0].message.content
    benchmark_df.loc[len(benchmark_df)] = {'subreddit': row['subreddit'], 'question': resp}
benchmark_df.drop_duplicates(subset=['question'], inplace=True)
benchmark_df.to_csv('benchmark_questions.csv', index=False)

In [ ]:
# format for benchmark part 2

# question, answer, difficulty
# on a scale of 1-10, how do people feel about the quality of the food at the University of Washington?, 8, hard